In [1]:
# Clear any logs from previous runs

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd

In [3]:
np.random.seed(0)

tf.config.optimizer.set_jit(True)
scaler = StandardScaler()

In [4]:
#algorithm for splitting the dataset into training and validation 
def split(X,Y,porcent): #porcent must be between 0 and 1, it is the asigned porcent to the training dataset.
    n=floor(porcent*len(X))
    index=random.sample(range(len(X)),n)
    X_learn=[]
    Y_learn=[]
    for i in index:
        X_learn.append(X[i])
        Y_learn.append(Y[i])
    X_val=np.delete(X,index, axis=0)
    Y_val=np.delete(Y,index, axis=0)
    
    X_learn=np.array(X_learn)
    Y_learn=np.array(Y_learn)
    return X_learn,Y_learn,X_val,Y_val

In [5]:
O_m=np.arange(0.1,0.51,0.01)
H_0=np.arange(66,81,1)
t=np.linspace(0,-12,50)

In [6]:
def RHS(Omega_i, lna, gamma=0):
    x, y, z, H = Omega_i
    #x, y, z = Omega_i
    pi = 3*x + 4*y
    return [x*(-3 + pi), y*(-4 + pi), z*pi, -0.5*H*pi]
    #return [x*(-3 + pi), y*(-4 + pi), z*pi]

def EDO(t,Om,H0):
    #t,Or,Om,Ol=X
    Or=0.0001
    Ol=1-Or-Om
    #H0 = 70.
    y0 = [Om, Or, Ol, H0]
    result = odeint(RHS, y0, t)
    return result

In [7]:
#lets generate the cartesian product between the intervals
Y0=[]
#este ciclo llena la lista fijando un Om y pasando todos los Or
for i in O_m:
    for j in H_0:
        Y0.extend(EDO(t,i,j))
Y0=np.array(Y0)

X0=[]
for Om in O_m:
    for H0 in H_0:
        for T in t:
            X0.append([T,Om,H0])
X0=np.array(X0)

In [8]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler(feature_range = (0,1))
Y2 = scaler.fit_transform(Y0)

In [9]:
#Now, here are the datasets
print('feactures= \n',X0)
print('\n')
print('labels= \n',Y2)

feactures= 
 [[  0.           0.1         66.        ]
 [ -0.24489796   0.1         66.        ]
 [ -0.48979592   0.1         66.        ]
 ...
 [-11.51020408   0.5         80.        ]
 [-11.75510204   0.5         80.        ]
 [-12.           0.5         80.        ]]


labels= 
 [[9.47515373e-02 0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [1.83608702e-01 1.41110961e-04 9.01997379e-01 1.62358493e-10]
 [3.22333902e-01 4.33937896e-04 7.48914062e-01 4.77144941e-10]
 ...
 [4.20164900e-02 9.58104047e-01 5.58399792e-11 3.78984027e-01]
 [3.19464665e-02 9.68145182e-01 5.73926563e-11 6.15280156e-01]
 [2.39153274e-02 9.76153281e-01 5.75109456e-11 1.00000000e+00]]


In [10]:
split_size = 0.8
X_train, Y_train, X_test, Y_test = split(X0, Y2, split_size)

### Hiperparámetros del modelo

In [11]:
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([1, 4, 8]))


HP_LAYERS =    hp.HParam('layers', hp.Discrete([2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-5,1e-4,1e-3]))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0,
                                   patience=6,
                                   restore_best_weights=True)]
# batch_size = 128
epochs = 50

In [12]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning2').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [13]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Input(shape=(int(X_train.shape[1]),)))
    model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(4, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=True, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [14]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [15]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning2/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 2, 'num_units': 50, 'learning_rate': 1e-05, 'batch_size': 1}
193/193 [==============================] - 0s 1ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Loss: 0.002899657469242811 Tiempo transcurrido: 1163.5126078128815

--- Starting trial: run-1
{'layers': 2, 'num_units': 50, 'learning_rate': 1e-05, 'batch_size': 4}
193/193 [==============================] - 0s 799us/step - loss: 0.0038 - mean_squared_error: 0.0038
Loss: 0.0037683527916669846 Tiempo transcurrido: 317.6881787776947

--- Starting trial: run-2
{'layers': 2, 'num_units': 50, 'learning_rate': 1e-05, 'batch_size': 8}
193/193 [==============================] - 0s 688us/step - loss: 0.0043 - mean_squared_error: 0.0043
Loss: 0.00425115367397666 Tiempo transcurrido: 152.47157073020935

--- Starting trial: run-3
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 1}
193/193 [==============================] - 0s 978us/step - loss: 3.2908e-04 - mean_squared_error: 3.29

193/193 [==============================] - 0s 1ms/step - loss: 0.0033 - mean_squared_error: 0.0033
Loss: 0.003341054543852806 Tiempo transcurrido: 511.1214790344238

--- Starting trial: run-20
{'layers': 2, 'num_units': 200, 'learning_rate': 1e-05, 'batch_size': 8}
193/193 [==============================] - 0s 1ms/step - loss: 0.0037 - mean_squared_error: 0.0037
Loss: 0.003660778049379587 Tiempo transcurrido: 249.26061153411865

--- Starting trial: run-21
{'layers': 2, 'num_units': 200, 'learning_rate': 0.0001, 'batch_size': 1}
193/193 [==============================] - 0s 1ms/step - loss: 1.7537e-04 - mean_squared_error: 1.7537e-04
Loss: 0.00017536831728648394 Tiempo transcurrido: 1636.4708721637726

--- Starting trial: run-22
{'layers': 2, 'num_units': 200, 'learning_rate': 0.0001, 'batch_size': 4}
193/193 [==============================] - 0s 886us/step - loss: 0.0010 - mean_squared_error: 0.0010
Loss: 0.0010366468923166394 Tiempo transcurrido: 374.9877259731293

--- Starting trial:

193/193 [==============================] - 0s 623us/step - loss: 0.0035 - mean_squared_error: 0.0035
Loss: 0.003459490602836013 Tiempo transcurrido: 108.6929395198822

--- Starting trial: run-39
{'layers': 3, 'num_units': 100, 'learning_rate': 0.0001, 'batch_size': 1}
193/193 [==============================] - 0s 878us/step - loss: 1.7030e-04 - mean_squared_error: 1.7030e-04
Loss: 0.00017029899754561484 Tiempo transcurrido: 791.1966662406921

--- Starting trial: run-40
{'layers': 3, 'num_units': 100, 'learning_rate': 0.0001, 'batch_size': 4}
193/193 [==============================] - 0s 580us/step - loss: 2.7453e-04 - mean_squared_error: 2.7453e-04
Loss: 0.00027452674112282693 Tiempo transcurrido: 216.54790568351746

--- Starting trial: run-41
{'layers': 3, 'num_units': 100, 'learning_rate': 0.0001, 'batch_size': 8}
193/193 [==============================] - 0s 588us/step - loss: 0.0015 - mean_squared_error: 0.0015
Loss: 0.001521717174910009 Tiempo transcurrido: 110.38120341300964

---

193/193 [==============================] - 0s 916us/step - loss: 1.2904e-04 - mean_squared_error: 1.2904e-04
Loss: 0.0001290350774070248 Tiempo transcurrido: 779.8782274723053

--- Starting trial: run-58
{'layers': 4, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
193/193 [==============================] - 0s 600us/step - loss: 3.0020e-04 - mean_squared_error: 3.0020e-04
Loss: 0.00030020088888704777 Tiempo transcurrido: 200.92625904083252

--- Starting trial: run-59
{'layers': 4, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
193/193 [==============================] - 0s 598us/step - loss: 7.3364e-04 - mean_squared_error: 7.3364e-04
Loss: 0.0007336366106756032 Tiempo transcurrido: 100.15522360801697

--- Starting trial: run-60
{'layers': 4, 'num_units': 50, 'learning_rate': 0.001, 'batch_size': 1}
193/193 [==============================] - 0s 912us/step - loss: 1.6669e-04 - mean_squared_error: 1.6669e-04
Loss: 0.0001666867610765621 Tiempo transcurrido: 767.49921

193/193 [==============================] - 0s 703us/step - loss: 1.9319e-04 - mean_squared_error: 1.9319e-04
Loss: 0.00019318897102493793 Tiempo transcurrido: 353.25851607322693

--- Starting trial: run-77
{'layers': 4, 'num_units': 200, 'learning_rate': 0.0001, 'batch_size': 8}
193/193 [==============================] - 0s 721us/step - loss: 4.9418e-04 - mean_squared_error: 4.9418e-04
Loss: 0.0004941769293509424 Tiempo transcurrido: 189.24344849586487

--- Starting trial: run-78
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 1}
193/193 [==============================] - 0s 1ms/step - loss: 4.6040e-04 - mean_squared_error: 4.6040e-04
Loss: 0.00046040076995268464 Tiempo transcurrido: 1017.5427732467651

--- Starting trial: run-79
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 4}
193/193 [==============================] - 0s 729us/step - loss: 1.6372e-04 - mean_squared_error: 1.6372e-04
Loss: 0.00016372051322832704 Tiempo transcurrido: 356.64

### Guardar datos

In [17]:
filename = "historial_ecsdiff_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[0,0], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [18]:
df

,Deep size,Num units,Learning rate,Batch size,MSE,Tiempo de ejecución
0,2,100,0.00001,8,0.004321,174.756558
1,2,50,0.00001,8,0.004251,152.471571
2,4,50,0.00001,8,0.003922,106.419520
3,3,50,0.00001,8,0.003789,95.301607
4,2,200,0.00001,8,0.003661,249.260612
5,3,100,0.00001,8,0.003459,108.692940
6,4,100,0.00001,8,0.003103,119.359766
7,3,200,0.00001,8,0.002953,157.012460
8,2,200,0.00010,8,0.002736,199.933555
9,4,200,0.00001,8,0.002611,188.570657


In [19]:
np.sum(df[["Tiempo de ejecución"]])/60

Tiempo de ejecución    62.07944
dtype: float64

In [20]:
# rm -rf /tmp/tb_logs/

### Now in terminal:
`python3 -m tensorboard.main --logdir='/home/isidro/Documents/github/neurapprox/logs/hparam_tuning'`

In [21]:
!kill 7439
%tensorboard --logdir logs/hparam_tuning2

/bin/sh: 1: kill: No such process



UsageError: Line magic function `%tensorboard` not found.
